# Challenge Problem 1: Don't Cross the Streams

In [1]:
import numpy as np
import random
from scipy.interpolate import splprep, splev
import plotly.graph_objects as go
from scipy.spatial.distance import cdist

Below are the design variables that you should change to impact the performance of the design and modify the placement of the tubes.

In [2]:
### These are your design variables.
#   Seven curves, with three changable control points in 3D
#   Changing them controls the shape of the conduits
#   and thus also the design objective function and
#   whether it satisfies the constraints.
design_variables = np.array([
[[0.9       , 0.82742166, 0.85945165],
 [0.9       , 0.66914808, 0.73243758],
 [0.84341331, 0.49880967, 0.58862069]],
[[0.59401668, 0.26060263, 0.22699443],
 [0.39994708, 0.16824367, 0.23736578],
 [0.19791618, 0.10115466, 0.11705611]],
[[0.10291537, 0.89292761, 0.19123033],
 [0.23319034, 0.79901707, 0.37907088],
 [0.31082852, 0.60186604, 0.55528925]],
[[0.5616076 , 0.45213118, 0.56217305],
 [0.71404417, 0.29137282, 0.38089722],
 [0.84848376, 0.1428194 , 0.18767941]],
[[0.17521742, 0.1       , 0.8947618 ],
 [0.3653463 , 0.18914621, 0.76755848],
 [0.52098123, 0.27852352, 0.65528529]],
[[0.52641041, 0.75642832, 0.61686231],
 [0.34394409, 0.78990153, 0.75838747],
 [0.16828042, 0.85060338, 0.86930358]],
[[0.7       , 0.74661119, 0.24820745],
 [0.1       , 0.49752301, 0.4983148 ],
 [0.6       , 0.24043227, 0.74348477]]])
print(design_variables.shape)

(7, 3, 3)


Everything else below is just helper/plotting code and you should not need to mess with it.

In [3]:

def bspline_curve(control_points, num_points=100):
    control_points = np.asarray(control_points)
    tck, u = splprep([control_points[:, 0], control_points[:, 1], control_points[:, 2]], s=0)
    u_new = np.linspace(u.min(), u.max(), num_points)
    x_new, y_new, z_new = splev(u_new, tck)
    return np.vstack((x_new, y_new, z_new)).T

def generate_noisy_intermediate_points(start_point, end_point, num_intermediate_points=3, noise_scale=0.01):
    # Generate linearly spaced intermediate points
    intermediate_points = np.linspace(start_point, end_point, num_intermediate_points+2)[1:-1]

    # Perturb the intermediate points with Gaussian noise
    noise = np.random.normal(scale=noise_scale, size=intermediate_points.shape)
    noisy_points = intermediate_points + noise

    # Make sure the control points stay positive within the unit cube
    noisy_points = np.clip(noisy_points, 0.1, 0.9)

    return noisy_points

def compute_closest_distance(curve, point):
    distances = cdist(curve, np.array([point]))
    return np.min(distances)

def compute_closest_distance_to_cube(curve):
    min_dist = np.inf
    for point in curve:
        if (0 <= point[0] <= 1) and (0 <= point[1] <= 1) and (0 <= point[2] <= 1):
            distances = np.min([point[0], 1-point[0], point[1], 1-point[1], point[2], 1-point[2]])
        else:
            distances = -np.min([abs(point[0]), abs(1-point[0]), abs(point[1]), abs(1-point[1]), abs(point[2]), abs(1-point[2])])
        if distances < min_dist:
            min_dist = distances
    return min_dist

def compute_total_length(curves):
    total_length = 0
    for curve in curves:
        diffs = np.diff(curve, axis=0)
        segment_lengths = np.linalg.norm(diffs, axis=1)
        total_length += np.sum(segment_lengths)
    return total_length

def compute_distance_between_curves(curve1, curve2):
    distances = cdist(curve1, curve2)
    distances_r = cdist(curve1, np.flip(curve2))
    return np.min([np.mean(distances),np.mean(distances_r)])

def are_curves_in_bounds(curves,eps=1e-2):
    for curve in curves:
        for point in curve:
            if not (0-eps <= point[0] <= 1+eps and 0-eps <= point[1] <= 1+eps and 0-eps <= point[2] <= 1+eps):
                # print(curve)
                # print(point)
                return False
    return True

def do_curves_intersect(curves, radius=0.05):
    for i, curve1 in enumerate(curves):
        for j, curve2 in enumerate(curves):
            if i != j:  # Avoid comparing a curve with itself
                # Now compute the distance and avoid the ends of the curves
                distances = cdist(curve1[5:-5,:], curve2[5:-5,:])
                min_distance = np.min(distances)
                if (min_distance - radius)<0:
                    return True, min_distance  # Curves intersect within radius
    return False, min_distance  # Curves do not intersect within radius


In [4]:
# Use for Generating the initial points, but then allow folks to modify
# Define the corners of a unit cube
corners = np.array([
    [0, 0, 0],
    [0, 0, 1],
    [0, 1, 0],
    [0, 1, 1],
    [1, 0, 0],
    [1, 0, 1],
    [1, 1, 0],
    [1, 1, 1]
])

np.random.seed(1)
np.random.shuffle(corners)
start_points = corners[0:4,:]
end_points = corners[4:,:]

In [5]:
# Generate three random sphere centers within the unit cube
num_spheres = 3
#sphere_centers = np.random.rand(num_spheres, 3)*.6+.2
sphere_centers = [[0.79942431, 0.34165339, 0.43794844],
                  [0.43274644, 0.60184762, 0.76132344],
                  [0.70778655, 0.38796411, 0.5147289 ]]
sphere_centers = [[0.29942431, 0.44165339, 0.43794844],
                  [0.63274644, 0.60184762, 0.76132344],
                  [0.70778655, 0.38796411, 0.5147289 ]]
sphere_radius = 1 / 15
sphere_colors = ['red', 'blue', 'green']
sphere_labels = ['Red Sphere', 'Blue Sphere', 'Green Sphere']

In [6]:
# Generate control points for each curve
curves = []
curve_colors = []
curve_labels = []
for i, (start, end) in enumerate(zip(start_points, end_points)):
    if i < num_spheres:
        sphere_center = sphere_centers[i]
        # Create two sets of control points: one from start to sphere center, another from sphere center to end
        # control_points1 = np.vstack([start,
        #                              generate_noisy_intermediate_points(start, sphere_center),
        #                              sphere_center])
        # control_points2 = np.vstack([sphere_center,
        #                              generate_noisy_intermediate_points(sphere_center, end),
        #                              end])
        control_points1 = np.vstack([start,
                                     design_variables[i*2],
                                     sphere_center])
        control_points2 = np.vstack([sphere_center,
                                     design_variables[i*2+1],
                                     end])
        curve_points1 = bspline_curve(control_points1, num_points=50)
        curve_points2 = bspline_curve(control_points2, num_points=50)
        curves.append(curve_points1)
        curves.append(curve_points2)
        curve_colors.append(sphere_colors[i])
        curve_colors.append(sphere_colors[i])
        curve_labels.append(f'{sphere_colors[i].capitalize()} Curve 1')
        curve_labels.append(f'{sphere_colors[i].capitalize()} Curve 2')
    else:
        # Generate intermediate control points randomly within the cube
        # control_points = np.vstack([start,
        #                             generate_noisy_intermediate_points(start, end),
        #                             end])
        control_points = np.vstack([start,
                                    design_variables[i*2],
                                    end])
        curve_points = bspline_curve(control_points, num_points=100)
        curves.append(curve_points)
        curve_colors.append('grey')
        curve_labels.append('Grey Curve')

In [7]:
# Compute the closest distance between the red curve and the blue sphere
red_curve = np.vstack([curves[i] for i in range(len(curves)) if curve_colors[i] == 'red'])
blue_sphere_center = sphere_centers[sphere_colors.index('blue')]
closest_distance_red_blue = compute_closest_distance(red_curve, blue_sphere_center)

# Compute the closest distance between the green curve and the sides of the unit cube
green_curve = np.vstack([curves[i] for i in range(len(curves)) if curve_colors[i] == 'green'])
closest_distance_green_cube = compute_closest_distance_to_cube(green_curve)

# Compute the total length of all curves
total_curve_length = compute_total_length(curves)

# Compute how far away the blue curve is to the green curve along its entire curve length
blue_curve = np.vstack([curves[i] for i in range(len(curves)) if curve_colors[i] == 'blue'])
distance_blue_green_curves = compute_distance_between_curves(blue_curve, green_curve)

# Check if all curves are within the unit cube
is_inside_cube = are_curves_in_bounds(curves)

# Check for curve intersections
intersecting = do_curves_intersect(curves)

# Print the results
# print(f"Closest distance between the red curve and the blue sphere: {closest_distance_red_blue}")
# print(f"Closest distance between the green curve and the sides of the unit cube: {closest_distance_green_cube}")
print(f"Total length of all curves: {total_curve_length}")
# print(f"Average distance between the blue curve and the green curve: {distance_blue_green_curves}")

# Overall Objective Function:
objective = 0
if(is_inside_cube and not intersecting):
    objective = (1-closest_distance_red_blue)+\
                (1-closest_distance_green_cube)+\
                5/total_curve_length+\
                distance_blue_green_curves
print(f"Overall Objective Function for Design (make high as possible): {objective}")

print(f"red_blue:{1-closest_distance_red_blue}")


Total length of all curves: 9.400119909545616
Overall Objective Function for Design (make high as possible): 0
red_blue:0.7301439800561553


In [8]:
# Create a 3D plot using Plotly
fig = go.Figure()

# Plot each curve
for i, (curve, color, label) in enumerate(zip(curves, curve_colors, curve_labels)):
    fig.add_trace(go.Scatter3d(
        x=curve[:, 0], y=curve[:, 1], z=curve[:, 2],
        mode='lines',
        name=label,
        line=dict(width=4, color=color)
    ))

# Plot the start and end points
for i in range(4):
    fig.add_trace(go.Scatter3d(
        x=[start_points[i][0]], y=[start_points[i][1]], z=[start_points[i][2]],
        mode='markers',
        name=f'Start {i+1}',
        marker=dict(size=6, color='black')
    ))
    fig.add_trace(go.Scatter3d(
        x=[end_points[i][0]], y=[end_points[i][1]], z=[end_points[i][2]],
        mode='markers',
        name=f'End {i+1}',
        marker=dict(size=6, color='black')
    ))

# Plot the spheres
for i, (center, color, label) in enumerate(zip(sphere_centers, sphere_colors, sphere_labels)):
    fig.add_trace(go.Scatter3d(
        x=[center[0]], y=[center[1]], z=[center[2]],
        mode='markers+text',
        name=label,
        marker=dict(size=10, color=color),
        text=[label],
        textposition="top center"
    ))

# Define the edges of the unit cube
cube_edges = [
    ([0, 1], [0, 0], [0, 0]),
    ([0, 1], [0, 0], [1, 1]),
    ([0, 1], [1, 1], [0, 0]),
    ([0, 1], [1, 1], [1, 1]),
    ([0, 0], [0, 1], [0, 0]),
    ([1, 1], [0, 1], [0, 0]),
    ([0, 0], [0, 1], [1, 1]),
    ([1, 1], [0, 1], [1, 1]),
    ([0, 0], [0, 0], [0, 1]),
    ([1, 1], [0, 0], [0, 1]),
    ([0, 0], [1, 1], [0, 1]),
    ([1, 1], [1, 1], [0, 1]),
]

# Plot the edges of the unit cube
for edge in cube_edges:
    fig.add_trace(go.Scatter3d(
        x=edge[0],
        y=edge[1],
        z=edge[2],
        mode='lines',
        line=dict(color='black', width=2),
        showlegend=False
    ))

# Update layout for better visualization
fig.update_layout(
    scene=dict(
        xaxis=dict(title='X-axis'),
        yaxis=dict(title='Y-axis'),
        zaxis=dict(title='Z-axis'),
    ),
    # title="Don't Cross the Streams!",
    margin=dict(l=0, r=0, b=0, t=40)
)

# fig.write_image("Initial_Design_Best.png", width=1920, height=1080, scale=2)

# Show the plot
fig.show()

# Print the objectives
print(f"Total length of all curves: {total_curve_length}")
print(f"Overall Objective Function for Design (make high as possible): {objective}")

Total length of all curves: 9.400119909545616
Overall Objective Function for Design (make high as possible): 0


## Solving the problem in full dimension

In [11]:
import scipy

shapes = [(50, 3)] * 6 + [(100, 3)]  # List of original shapes
def curve_generator(design_variables):
    # Generate control points for each curve
    curves = []
    curve_colors = []
    curve_labels = []
    for i, (start, end) in enumerate(zip(start_points, end_points)):
        if i < num_spheres:
            sphere_center = sphere_centers[i]
            # Create two sets of control points: one from start to sphere center, another from sphere center to end
            # control_points1 = np.vstack([start,
            #                              generate_noisy_intermediate_points(start, sphere_center),
            #                              sphere_center])
            # control_points2 = np.vstack([sphere_center,
            #                              generate_noisy_intermediate_points(sphere_center, end),
            #                              end])
            control_points1 = np.vstack([start,
                                        design_variables[i*2],
                                        sphere_center])
            control_points2 = np.vstack([sphere_center,
                                        design_variables[i*2+1],
                                        end])
            curve_points1 = bspline_curve(control_points1, num_points=50)
            curve_points2 = bspline_curve(control_points2, num_points=50)
            curves.append(curve_points1)
            curves.append(curve_points2)
            curve_colors.append(sphere_colors[i])
            curve_colors.append(sphere_colors[i])
            curve_labels.append(f'{sphere_colors[i].capitalize()} Curve 1')
            curve_labels.append(f'{sphere_colors[i].capitalize()} Curve 2')
        else:
            # Generate intermediate control points randomly within the cube
            # control_points = np.vstack([start,
            #                             generate_noisy_intermediate_points(start, end),
            #                             end])
            control_points = np.vstack([start,
                                        design_variables[i*2],
                                        end])
            curve_points = bspline_curve(control_points, num_points=100)
            curves.append(curve_points)
            curve_colors.append('grey')
            curve_labels.append('Grey Curve')
    return curves

def objective_function(x):
    x= x.reshape(7, 3, 3)
    curves = curve_generator(x)

    # Compute the closest distance between the red curve and the blue sphere
    red_curve = np.vstack([curves[i] for i in range(len(curves)) if curve_colors[i] == 'red'])
    blue_sphere_center = sphere_centers[sphere_colors.index('blue')]
    closest_distance_red_blue = compute_closest_distance(red_curve, blue_sphere_center)

    # Compute the closest distance between the green curve and the sides of the unit cube
    green_curve = np.vstack([curves[i] for i in range(len(curves)) if curve_colors[i] == 'green'])
    closest_distance_green_cube = compute_closest_distance_to_cube(green_curve)

    # Compute the total length of all curves
    total_curve_length = compute_total_length(curves)

    # Compute how far away the blue curve is to the green curve along its entire curve length
    blue_curve = np.vstack([curves[i] for i in range(len(curves)) if curve_colors[i] == 'blue'])
    distance_blue_green_curves = compute_distance_between_curves(blue_curve, green_curve)

    # Check if all curves are within the unit cube
    is_inside_cube = are_curves_in_bounds(curves)

    # Check for curve intersections
    intersecting, min_dis = do_curves_intersect(curves)

    # Overall Objective Function:
    objective = 100
    if(is_inside_cube and not intersecting):
        objective = -((closest_distance_red_blue)+\
                    (1-closest_distance_green_cube)+\
                    5/total_curve_length+\
                    distance_blue_green_curves)
    else:    
        objective = - objective + total_curve_length - 1*(min_dis-0.05)
    return objective

def intersecting_cons(x):
    x= x.reshape(7, 3, 3)
    curves = curve_generator(x)
    hit,_= do_curves_intersect(curves)
    if hit:
        return -10000
    else:
        return 0
    
def bounds_cons(x):
    x= x.reshape(7, 3, 3)
    curves = curve_generator(x)
    if are_curves_in_bounds(curves):
        return 0
    else:
        return -10000

cons = ({'type': 'ineq', 'fun': intersecting_cons},
        {'type': 'ineq', 'fun': bounds_cons})

X0 = design_variables.reshape(-1)
sol=scipy.optimize.minimize(objective_function,x0=X0, constraints=cons, method= 'trust-constr')
# sol=scipy.optimize.minimize(objective_function,x0=X0, constraints=cons, method= 'SLSQP')

x_sol = sol.x
x_sol = x_sol.reshape(7, 3, 3)
curves_sol = curve_generator(x_sol)

In [12]:
# Compute the closest distance between the red curve and the blue sphere
red_curve = np.vstack([curves_sol[i] for i in range(len(curves_sol)) if curve_colors[i] == 'red'])
blue_sphere_center = sphere_centers[sphere_colors.index('blue')]
closest_distance_red_blue = compute_closest_distance(red_curve, blue_sphere_center)

# Compute the closest distance between the green curve and the sides of the unit cube
green_curve = np.vstack([curves_sol[i] for i in range(len(curves_sol)) if curve_colors[i] == 'green'])
closest_distance_green_cube = compute_closest_distance_to_cube(green_curve)

# Compute the total length of all curves
total_curve_length = compute_total_length(curves_sol)

# Compute how far away the blue curve is to the green curve along its entire curve length
blue_curve = np.vstack([curves_sol[i] for i in range(len(curves_sol)) if curve_colors[i] == 'blue'])
distance_blue_green_curves = compute_distance_between_curves(blue_curve, green_curve)

# Check if all curves are within the unit cube
is_inside_cube = are_curves_in_bounds(curves_sol)

# Check for curve intersections
intersecting,_ = do_curves_intersect(curves_sol)

# Print the results
# print(f"Closest distance between the red curve and the blue sphere: {closest_distance_red_blue}")
# print(f"Closest distance between the green curve and the sides of the unit cube: {closest_distance_green_cube}")
print(f"Total length of all curves: {total_curve_length}")
# print(f"Average distance between the blue curve and the green curve: {distance_blue_green_curves}")

# Overall Objective Function:
objective = 0
if(is_inside_cube and not intersecting):
    objective = (1-closest_distance_red_blue)+\
                (1-closest_distance_green_cube)+\
                5/total_curve_length+\
                distance_blue_green_curves
print(f"Overall Objective Function for Design (make high as possible): {objective}")

Total length of all curves: 7.60172534826504
Overall Objective Function for Design (make high as possible): 2.832694660253166


In [13]:
# Create a 3D plot using Plotly
fig = go.Figure()

# Plot each curve
for i, (curve, color, label) in enumerate(zip(curves_sol, curve_colors, curve_labels)):
    fig.add_trace(go.Scatter3d(
        x=curve[:, 0], y=curve[:, 1], z=curve[:, 2],
        mode='lines',
        name=label,
        line=dict(width=4, color=color)
    ))

# Plot the start and end points
for i in range(4):
    fig.add_trace(go.Scatter3d(
        x=[start_points[i][0]], y=[start_points[i][1]], z=[start_points[i][2]],
        mode='markers',
        name=f'Start {i+1}',
        marker=dict(size=6, color='black')
    ))
    fig.add_trace(go.Scatter3d(
        x=[end_points[i][0]], y=[end_points[i][1]], z=[end_points[i][2]],
        mode='markers',
        name=f'End {i+1}',
        marker=dict(size=6, color='black')
    ))

# Plot the spheres
for i, (center, color, label) in enumerate(zip(sphere_centers, sphere_colors, sphere_labels)):
    fig.add_trace(go.Scatter3d(
        x=[center[0]], y=[center[1]], z=[center[2]],
        mode='markers+text',
        name=label,
        marker=dict(size=10, color=color),
        text=[label],
        textposition="top center"
    ))

# May Break things
# Plot the control points
# for i, (center) in enumerate(design_variables):
#     for j, curve in enumerate(design_variables[i]):
#         fig.add_trace(go.Scatter3d(
#             x=[center[0]], y=[center[1]], z=[center[2]],
#             mode='markers',
#             marker=dict(size=3, color='purple')
#         ))

# Define the edges of the unit cube
cube_edges = [
    ([0, 1], [0, 0], [0, 0]),
    ([0, 1], [0, 0], [1, 1]),
    ([0, 1], [1, 1], [0, 0]),
    ([0, 1], [1, 1], [1, 1]),
    ([0, 0], [0, 1], [0, 0]),
    ([1, 1], [0, 1], [0, 0]),
    ([0, 0], [0, 1], [1, 1]),
    ([1, 1], [0, 1], [1, 1]),
    ([0, 0], [0, 0], [0, 1]),
    ([1, 1], [0, 0], [0, 1]),
    ([0, 0], [1, 1], [0, 1]),
    ([1, 1], [1, 1], [0, 1]),
]

# Plot the edges of the unit cube
for edge in cube_edges:
    fig.add_trace(go.Scatter3d(
        x=edge[0],
        y=edge[1],
        z=edge[2],
        mode='lines',
        line=dict(color='black', width=2),
        showlegend=False
    ))

# Update layout for better visualization
fig.update_layout(
    scene=dict(
        xaxis=dict(title='X-axis'),
        yaxis=dict(title='Y-axis'),
        zaxis=dict(title='Z-axis'),
    ),
    # title="Don't Cross the Streams!",
    margin=dict(l=0, r=0, b=0, t=40)
)


# Show the plot
fig.show()

# Print the objectives
print(f"Total length of all curves: {total_curve_length}")
print(f"Overall Objective Function for Design (make high as possible): {objective}")

Total length of all curves: 7.60172534826504
Overall Objective Function for Design (make high as possible): 2.832694660253166


## Solving the problem in reduced dimension

In [14]:
design_variables = np.array([
[[0.8248560775, 0.8604133475, 0.85948711],
 [0.649712155, 0.720826695, 0.71897422],
 [0.4745682325, 0.5812400424999999, 0.57846133]],
[[0.22456823250000002, 0.3312400425, 0.32846133],
 [0.149712155, 0.220826695, 0.21897422],
 [0.0748560775, 0.11041334749999998, 0.10948711]],
[[0.15818661, 0.900461905, 0.19033086],
 [0.31637322, 0.80092381, 0.38066172],
 [0.47455983, 0.701385715, 0.57099258]],
[[0.72455983, 0.451385715, 0.57099258],
 [0.81637322, 0.30092381, 0.38066172],
 [0.90818661, 0.150461905, 0.19033086]],
[[0.1769466375, 0.0969910275, 0.878682225],
 [0.353893275, 0.193982055, 0.7573644500000001],
 [0.5308399125000001, 0.2909730825, 0.636046675]],
[[0.5308399125000001, 0.5409730825, 0.636046675],
 [0.353893275, 0.693982055, 0.7573644500000001],
 [0.17694663749999995, 0.8469910275, 0.878682225]],
[[1       , 0.75, 0.25],
 [1       , 0.5, 0.5 ],
 [1       , 0.25, 0.75]]])

In [15]:
import scipy

shapes = [(50, 3)] * 6 + [(100, 3)]  # List of original shapes
def curve_generator(design_variables):
    # Generate control points for each curve
    curves = []
    curve_colors = []
    curve_labels = []
    for i, (start, end) in enumerate(zip(start_points, end_points)):
        if i < num_spheres:
            sphere_center = sphere_centers[i]
            # Create two sets of control points: one from start to sphere center, another from sphere center to end
            # control_points1 = np.vstack([start,
            #                              generate_noisy_intermediate_points(start, sphere_center),
            #                              sphere_center])
            # control_points2 = np.vstack([sphere_center,
            #                              generate_noisy_intermediate_points(sphere_center, end),
            #                              end])
            control_points1 = np.vstack([start,
                                        design_variables[i*2],
                                        sphere_center])
            control_points2 = np.vstack([sphere_center,
                                        design_variables[i*2+1],
                                        end])
            curve_points1 = bspline_curve(control_points1, num_points=50)
            curve_points2 = bspline_curve(control_points2, num_points=50)
            curves.append(curve_points1)
            curves.append(curve_points2)
            curve_colors.append(sphere_colors[i])
            curve_colors.append(sphere_colors[i])
            curve_labels.append(f'{sphere_colors[i].capitalize()} Curve 1')
            curve_labels.append(f'{sphere_colors[i].capitalize()} Curve 2')
        else:
            # Generate intermediate control points randomly within the cube
            # control_points = np.vstack([start,
            #                             generate_noisy_intermediate_points(start, end),
            #                             end])
            control_points = np.vstack([start,
                                        design_variables[i*2],
                                        end])
            curve_points = bspline_curve(control_points, num_points=100)
            curves.append(curve_points)
            curve_colors.append('grey')
            curve_labels.append('Grey Curve')
    return curves

def objective_function(x):
    # x= x.reshape(7, 3, 3)
    # x= x.reshape(4, 3, 3)
    x= x.reshape(3, 3, 3)
    x1 = np.append([x[0,:,:]],design_variables[1:3,:,:],axis=0)
    x1 = np.append(x1, [x[1,:,:]], axis=0)
    x1 = np.append(x1,design_variables[4:5,:,:],axis=0)
    x1 = np.append(x1, [x[2,:,:]], axis=0)
    # x= np.append(x,design_variables[4:,:,:],axis =0)
    x= np.append(x1,design_variables[6:,:,:],axis =0)
    curves = curve_generator(x)

    # Compute the closest distance between the red curve and the blue sphere
    red_curve = np.vstack([curves[i] for i in range(len(curves)) if curve_colors[i] == 'red'])
    blue_sphere_center = sphere_centers[sphere_colors.index('blue')]
    closest_distance_red_blue = compute_closest_distance(red_curve, blue_sphere_center)

    # Compute the closest distance between the green curve and the sides of the unit cube
    green_curve = np.vstack([curves[i] for i in range(len(curves)) if curve_colors[i] == 'green'])
    closest_distance_green_cube = compute_closest_distance_to_cube(green_curve)

    # Compute the total length of all curves
    total_curve_length = compute_total_length(curves)

    # Compute how far away the blue curve is to the green curve along its entire curve length
    blue_curve = np.vstack([curves[i] for i in range(len(curves)) if curve_colors[i] == 'blue'])
    distance_blue_green_curves = compute_distance_between_curves(blue_curve, green_curve)

    # Check if all curves are within the unit cube
    is_inside_cube = are_curves_in_bounds(curves)

    # Check for curve intersections
    intersecting, min_dis = do_curves_intersect(curves)

    # Overall Objective Function:
    objective = 100
    if(is_inside_cube and not intersecting):
        objective = -((closest_distance_red_blue)+\
                    (1-closest_distance_green_cube)+\
                    5/total_curve_length+\
                    distance_blue_green_curves)
    else:    
        objective = - objective + total_curve_length - 1*(min_dis-0.05)
    return objective

def intersecting_cons(x):
    # x= x.reshape(7, 3, 3)
    # x= x.reshape(4, 3, 3)
    # x= np.append(x,design_variables[4:,:,:],axis =0)
    x= x.reshape(3, 3, 3)
    x1 = np.append([x[0,:,:]],design_variables[1:3,:,:],axis=0)
    x1 = np.append(x1, [x[1,:,:]], axis=0)
    x1 = np.append(x1,design_variables[4:5,:,:],axis=0)
    x1 = np.append(x1, [x[2,:,:]], axis=0)
    # x= np.append(x,design_variables[4:,:,:],axis =0)
    x= np.append(x1,design_variables[6:,:,:],axis =0)
    curves = curve_generator(x)
    hit,_= do_curves_intersect(curves)
    if hit:
        return -10000
    else:
        return 0
    
def bounds_cons(x):
    x= x.reshape(3, 3, 3)
    x1 = np.append([x[0,:,:]],design_variables[1:3,:,:],axis=0)
    x1 = np.append(x1, [x[1,:,:]], axis=0)
    x1 = np.append(x1,design_variables[4:5,:,:],axis=0)
    x1 = np.append(x1, [x[2,:,:]], axis=0)
    # x= np.append(x,design_variables[4:,:,:],axis =0)
    x= np.append(x1,design_variables[6:,:,:],axis =0)
    curves = curve_generator(x)
    if are_curves_in_bounds(curves):
        return 0
    else:
        return -10000

cons = ({'type': 'ineq', 'fun': intersecting_cons},
        {'type': 'ineq', 'fun': bounds_cons})

X0 = design_variables.reshape(-1)
# X0 = X0[0:63-27]
X1 = np.append(X0[0:9],X0[27:36])
X0 = np.append(X1,X0[45:63-9])

sol=scipy.optimize.minimize(objective_function,x0=X0, constraints=cons, method= 'trust-constr')

x_sol = sol.x
x_sol= x_sol.reshape(3, 3, 3)
x1_sol = np.append([x_sol[0,:,:]],design_variables[1:3,:,:],axis=0)
x1_sol = np.append(x1_sol, [x_sol[1,:,:]], axis=0)
x1_sol = np.append(x1_sol,design_variables[4:5,:,:],axis=0)
x1_sol = np.append(x1_sol, [x_sol[2,:,:]], axis=0)
x_sol= np.append(x1_sol,design_variables[6:,:,:],axis =0)
curves_sol = curve_generator(x_sol)

In [16]:
# Compute the closest distance between the red curve and the blue sphere
red_curve = np.vstack([curves_sol[i] for i in range(len(curves_sol)) if curve_colors[i] == 'red'])
blue_sphere_center = sphere_centers[sphere_colors.index('blue')]
closest_distance_red_blue = compute_closest_distance(red_curve, blue_sphere_center)

# Compute the closest distance between the green curve and the sides of the unit cube
green_curve = np.vstack([curves_sol[i] for i in range(len(curves_sol)) if curve_colors[i] == 'green'])
closest_distance_green_cube = compute_closest_distance_to_cube(green_curve)

# Compute the total length of all curves
total_curve_length = compute_total_length(curves_sol)

# Compute how far away the blue curve is to the green curve along its entire curve length
blue_curve = np.vstack([curves_sol[i] for i in range(len(curves_sol)) if curve_colors[i] == 'blue'])
distance_blue_green_curves = compute_distance_between_curves(blue_curve, green_curve)

# Check if all curves are within the unit cube
is_inside_cube = are_curves_in_bounds(curves_sol)

# Check for curve intersections
intersecting,_ = do_curves_intersect(curves_sol)

# Print the results
# print(f"Closest distance between the red curve and the blue sphere: {closest_distance_red_blue}")
# print(f"Closest distance between the green curve and the sides of the unit cube: {closest_distance_green_cube}")
print(f"Total length of all curves: {total_curve_length}")
# print(f"Average distance between the blue curve and the green curve: {distance_blue_green_curves}")

# Overall Objective Function:
objective = 0
if(is_inside_cube and not intersecting):
    objective = (1-closest_distance_red_blue)+\
                (1-closest_distance_green_cube)+\
                5/total_curve_length+\
                distance_blue_green_curves
print(f"Overall Objective Function for Design (make high as possible): {objective}")

Total length of all curves: 7.712212423812835
Overall Objective Function for Design (make high as possible): 2.804733693161932


In [17]:
# Create a 3D plot using Plotly
fig = go.Figure()

# Plot each curve
for i, (curve, color, label) in enumerate(zip(curves_sol, curve_colors, curve_labels)):
    fig.add_trace(go.Scatter3d(
        x=curve[:, 0], y=curve[:, 1], z=curve[:, 2],
        mode='lines',
        name=label,
        line=dict(width=4, color=color)
    ))

# Plot the start and end points
for i in range(4):
    fig.add_trace(go.Scatter3d(
        x=[start_points[i][0]], y=[start_points[i][1]], z=[start_points[i][2]],
        mode='markers',
        name=f'Start {i+1}',
        marker=dict(size=6, color='black')
    ))
    fig.add_trace(go.Scatter3d(
        x=[end_points[i][0]], y=[end_points[i][1]], z=[end_points[i][2]],
        mode='markers',
        name=f'End {i+1}',
        marker=dict(size=6, color='black')
    ))

# Plot the spheres
for i, (center, color, label) in enumerate(zip(sphere_centers, sphere_colors, sphere_labels)):
    fig.add_trace(go.Scatter3d(
        x=[center[0]], y=[center[1]], z=[center[2]],
        mode='markers+text',
        name=label,
        marker=dict(size=10, color=color),
        text=[label],
        textposition="top center"
    ))

# May Break things
# Plot the control points
# for i, (center) in enumerate(design_variables):
#     for j, curve in enumerate(design_variables[i]):
#         fig.add_trace(go.Scatter3d(
#             x=[center[0]], y=[center[1]], z=[center[2]],
#             mode='markers',
#             marker=dict(size=3, color='purple')
#         ))

# Define the edges of the unit cube
cube_edges = [
    ([0, 1], [0, 0], [0, 0]),
    ([0, 1], [0, 0], [1, 1]),
    ([0, 1], [1, 1], [0, 0]),
    ([0, 1], [1, 1], [1, 1]),
    ([0, 0], [0, 1], [0, 0]),
    ([1, 1], [0, 1], [0, 0]),
    ([0, 0], [0, 1], [1, 1]),
    ([1, 1], [0, 1], [1, 1]),
    ([0, 0], [0, 0], [0, 1]),
    ([1, 1], [0, 0], [0, 1]),
    ([0, 0], [1, 1], [0, 1]),
    ([1, 1], [1, 1], [0, 1]),
]

# Plot the edges of the unit cube
for edge in cube_edges:
    fig.add_trace(go.Scatter3d(
        x=edge[0],
        y=edge[1],
        z=edge[2],
        mode='lines',
        line=dict(color='black', width=2),
        showlegend=False
    ))

# Update layout for better visualization
fig.update_layout(
    scene=dict(
        xaxis=dict(title='X-axis'),
        yaxis=dict(title='Y-axis'),
        zaxis=dict(title='Z-axis'),
    ),
    # title="Don't Cross the Streams!",
    margin=dict(l=0, r=0, b=0, t=40)
)


# Show the plot
fig.show()

# Print the objectives
print(f"Total length of all curves: {total_curve_length}")
print(f"Overall Objective Function for Design (make high as possible): {objective}")

Total length of all curves: 7.712212423812835
Overall Objective Function for Design (make high as possible): 2.804733693161932
